In [8]:
import os
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, HeatMap
import numpy as np

In [2]:
vci_df = pd.read_csv(os.path.join("data", "vci_constituency.csv"))

In [3]:
vci_df.head()

,constituency_name,counstituency_code,vci_aug_16,vci_sep_16,vci_nov_16,vci_dec_16,vci_jan_17
0,GANZE,15,11.85,14.66,2.80,-10.82,2.64
1,KILIFI NORTH,11,12.36,16.56,-0.08,-23.20,-13.79
2,KALOLENI,13,14.14,14.65,-6.57,-23.91,-15.14
3,MALINDI,16,16.65,16.16,-1.17,-16.81,-7.23
4,KINANGO,10,19.17,17.62,3.52,-4.82,3.68


In [4]:
january_vci = {k: v for (k, v) in zip([row.counstituency_code for index, row in vci_df.iterrows()],
                                     [row.vci_jan_17 for index, row in vci_df.iterrows()])}
january_vci 

{7: 13.76,
 8: 4.7,
 9: 5.15,
 10: 3.68,
 11: -13.79,
 12: 2.13,
 13: -15.14,
 14: -16.62,
 15: 2.64,
 16: -7.23,
 17: -2.15,
 18: 9.2,
 19: 15.25,
 20: 20.42,
 21: 2.85,
 22: -13.26,
 23: 19.19,
 24: 25.26,
 25: 16.27,
 26: 26.05,
 27: 18.34,
 28: 17.19,
 29: 11.94,
 30: 14.04,
 31: 16.31,
 32: 7.87,
 33: 16.05,
 34: 22.15,
 35: 23.35,
 36: 7.66,
 37: 22.15,
 38: 12.06,
 39: 17.06,
 40: 18.12,
 41: 15.72,
 42: 25.81,
 43: 14.0,
 44: 18.69,
 45: 16.55,
 46: 8.92,
 47: 16.16,
 48: 11.02,
 49: 18.77,
 50: 21.39,
 51: 38.28,
 52: 35.14,
 53: 37.63,
 54: 23.99,
 55: 34.76,
 56: 22.81,
 57: 35.27,
 58: 35.96,
 59: 45.22,
 60: 45.35,
 61: 47.93,
 62: 23.65,
 63: 53.12,
 64: 60.09,
 65: 44.5,
 66: 41.48,
 67: 30.41,
 68: 34.86,
 69: 35.55,
 70: 33.97,
 71: 37.11,
 72: 42.87,
 73: 35.83,
 74: 29.99,
 83: 47.35,
 84: 40.25,
 85: 48.5,
 86: 45.89,
 87: 42.76,
 88: 41.33,
 94: 38.93,
 95: 44.43,
 96: 38.43,
 97: 40.66,
 98: 31.89,
 99: 24.92,
 123: 10.86,
 124: 13.47,
 125: 26.82,
 126: 17.48,
 1

In [5]:
constituencies_gdf = gpd.GeoDataFrame.from_file(os.path.join("data", "constituencies.geojson"))
constituencies_gdf.head()

,CONSTITUEN,CONST_CODE,geometry
0,KAMUKUNJI,288.0,POLYGON ((36.87595372679757 -1.262533413477499...
1,WESTLANDS,274.0,POLYGON ((36.81926300479753 -1.206434634477475...
2,DAGORETTI NORTH,275.0,POLYGON ((36.79214315479756 -1.260847108477478...
3,ROYSAMBU,279.0,POLYGON ((36.90578043879756 -1.161375592477484...
4,STAREHE,289.0,POLYGON ((36.84274500879754 -1.264599326477466...


In [7]:
styles = []

for row in constituencies_gdf["CONST_CODE"]:
    try:
        vci_value = january_vci[row]
        if vci_value >= 50:
            styles.append({"fillColor": "#0A9500", "weight": 1, "color": "#000000"})
        elif vci_value >= 35:
            styles.append({"fillColor": "#79C573", "weight": 1, "color": "#000000"})
        elif vci_value >= 21:
            styles.append({"fillColor": "#FFD819", "weight": 1, "color": "#000000"})
        elif vci_value >= 10:
            styles.append({"fillColor": "#FF8B8B", "weight": 1, "color": "#000000"})
        else:
            styles.append({"fillColor": "#FF0000", "weight": 1, "color": "#000000"})
    except KeyError:
        styles.append({"fillColor": "#424242", "weight": 1, "color": "#000000"})

constituencies_gdf["style"] = styles
constituencies_gdf.head()

,CONSTITUEN,CONST_CODE,geometry,style
0,KAMUKUNJI,288.0,POLYGON ((36.87595372679757 -1.262533413477499...,"{'weight': 1, 'color': '#000000', 'fillColor':..."
1,WESTLANDS,274.0,POLYGON ((36.81926300479753 -1.206434634477475...,"{'weight': 1, 'color': '#000000', 'fillColor':..."
2,DAGORETTI NORTH,275.0,POLYGON ((36.79214315479756 -1.260847108477478...,"{'weight': 1, 'color': '#000000', 'fillColor':..."
3,ROYSAMBU,279.0,POLYGON ((36.90578043879756 -1.161375592477484...,"{'weight': 1, 'color': '#000000', 'fillColor':..."
4,STAREHE,289.0,POLYGON ((36.84274500879754 -1.264599326477466...,"{'weight': 1, 'color': '#000000', 'fillColor':..."


In [44]:
vci_map = folium.Map([
    np.mean([row.geometry.centroid.y for index, row in constituencies_gdf.iterrows()]), 
    np.mean([row.geometry.centroid.x for index, row in constituencies_gdf.iterrows()])], 
    zoom_start = 6,
    tiles = "Cartodb Positron")

In [26]:
folium.GeoJson(constituencies_gdf).add_to(vci_map)

vci_map.add_child(MarkerCluster(locations=list(zip([row.geometry.centroid.y for index, row in constituencies_gdf.iterrows()], 
                                                   [row.geometry.centroid.x  for index, row in constituencies_gdf.iterrows()])), 
                                popups=[row.CONSTITUEN for index, row in constituencies_gdf.iterrows()]))

vci_map.save(os.path.join("html_files", "const_with_markers.html"))